In [1]:
import networkx as nx
import snap
import csv
import time
import numpy as np
import torch
import pandas as pd
import pickle

In [28]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
def extractSnapGraph(filename):
    NetworkXGraph = nx.readwrite.gexf.read_gexf(filename)
    newFileName = filename.split('.')
    finalFileName = newFileName[0] + '.edges'
    print finalFileName
    nx.write_edgelist(NetworkXGraph, finalFileName)
    SNAPGraph = snap.LoadEdgeList(snap.PUNGraph, finalFileName, 0, 1)
    return SNAPGraph

In [4]:
def extractNetworkXGraph(filename):
    NetworkXGraph = nx.readwrite.gexf.read_gexf(filename)
    return NetworkXGraph

In [5]:
def load_dictionary(file_name):
    with open(file_name + '.pkl', 'rb') as current_file:
        return pickle.load(current_file)

In [6]:
dictGenresToIds = load_dictionary('genre_to_ids')
dictIdToGenres = load_dictionary('id_to_genres')
dictIdToMovieName = load_dictionary('id_to_movie_name')

In [7]:
genres = []
for key, value in dictGenresToIds.iteritems():
    genres.append(key)

In [8]:
genres_to_ids = {}
for i in range(len(genres)):
    genres_to_ids[genres[i]] = i

In [9]:
new_dictIdToGenres = {}
for movie_id, genres_list in dictIdToGenres.iteritems():
    if movie_id not in new_dictIdToGenres.iterkeys():
        new_dictIdToGenres[movie_id] = []
    for i in range(len(genres_list)):
        genre = genres_list[i]
        new_dictIdToGenres[movie_id].append(genres_to_ids[genre])

In [10]:
new_dictIdToBinaryGenres = {} # id maps to a bit vector
for movie_id, genres_ids_list in new_dictIdToGenres.iteritems():
    bit_vector = []
    for i in range(len(genres)):
        if i in genres_ids_list:
            bit_vector.append(1)
        else:
            bit_vector.append(0)
    new_dictIdToBinaryGenres[movie_id] = bit_vector

In [11]:
new_dictGenresToIds = {}
for genre, ids_list in dictGenresToIds.iteritems():
    genre_id = genres_to_ids[genre]
    new_dictGenresToIds[genre_id] = ids_list

In [12]:
new_df = pd.read_csv('featureMatrix.csv', sep=' ')

In [13]:
X_data = []
Y_data = []
for index, row in new_df.iterrows():
    movie_id = row[0]
    if movie_id in new_dictIdToBinaryGenres.iterkeys():
        genres_list = new_dictIdToBinaryGenres[movie_id]
        Y_data.append(genres_list)
    else:
        continue
    new_movie = []
    for i in range(1, len(row)):
        new_movie.append(row[i])
    X_data.append(new_movie)

In [14]:
X_data = np.asarray(X_data)
Y_data = np.asarray(Y_data)

In [15]:
X_train = X_data[0:700]
Y_train = Y_data[0:700]

In [16]:
X_test = X_data[700:-1]
Y_test = Y_data[700:-1]

In [17]:
mlp = MLPClassifier(activation='tanh', solver='adam', alpha=1e-2, batch_size = 10,
                    learning_rate = 'invscaling', tol = 0.000001,
                    hidden_layer_sizes=(100, 2), max_iter = 1000, random_state=1)

In [18]:
mlp.fit(X_train, Y_train)

MLPClassifier(activation='tanh', alpha=0.01, batch_size=10, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 2), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=1e-06, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [19]:
score = mlp.score(X_test, Y_test) # num that matches exactly

In [20]:
Y_mlp_result = mlp.predict(X_test)

In [21]:
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
count = 0
for i in range(len(Y_mlp_result)):
    for j in range(len(Y_mlp_result[i])):
        count += 1
        if Y_test[i][j] == 1 and Y_mlp_result[i][j] == 1:
            true_positive += 1
        if Y_test[i][j] == 0 and Y_mlp_result[i][j] == 0:
            true_negative += 1
        if Y_test[i][j] == 1 and Y_mlp_result[i][j] == 0:
            false_negative += 1
        if Y_test[i][j] == 0 and Y_mlp_result[i][j] == 1:
            false_positive += 1

In [22]:
right = true_negative + true_positive
total = true_positive + true_negative + false_positive + false_negative

In [23]:
print float(right)/total

0.752777777778


In [24]:
fpr = float(false_positive) / (false_positive + true_negative)
tpr = float(true_positive) / (true_positive + false_negative)

In [25]:
y = np.array([1, 1, 2, 2])
pred = np.array([0.1, 0.4, 0.35, 0.8])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)

In [29]:
print roc_auc_score(Y_test, Y_mlp_result)

0.506656346749


In [ ]:
new_df = pd.read_csv('featureMatrix.csv', sep=',')

In [47]:
movies = []
for movie_name in dictIdToMovieName.itervalues():
    movies.append(movie_name)

In [39]:
movie_df = pd.read_csv('tmdb_5000_movies.csv', sep=',')

In [49]:
count = 0
for i in range(len(movie_df)):
    if movie_df.loc[i]['title'] in movies:
        count += 1

/Users/qldo18/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [50]:
count

523

In [ ]:
budget, popularity, runtime, revenue, vote_average, vote_count # production_company